<a href="https://colab.research.google.com/github/nlongcn/examples/blob/master/bert_semantic_search_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers==2.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Connect to Gmail API

In [ ]:
# already installed by default i believe
! pip install --upgrade google-api-python-client==2.83.0 google-auth==2.16.3 google-auth-httplib2==0.1.0 google-auth-oauthlib==0.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError
import google.auth.exceptions
from googleapiclient.discovery import build

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json')
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', ['https://www.googleapis.com/auth/gmail.readonly'])
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())
return creds

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=661640239003-5ncgrgactghtf48on2jnm9br9oqdug56.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A34021%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=1OZJVtH73gwxH80uO30YydPAeez6I9&access_type=offline


In [ ]:
emails = [] # indvidiual raw emails stored
CUT_OFF = 10000 # maximum number of emails to be downloaded
try:
  service = build("gmail", "v1", credentials=creds)
  gmail_messages = service.users().messages()
  
  has_next_token = True
  next_token = None
  count = 0
  while(has_next_token):
    results = gmail_messages.list(userId='me', pageToken=next_token).execute()
    messages = results["messages"]
    if "nextPageToken" in results:
      next_token = results["nextPageToken"]
    else:
      next_token = None
      has_next_token = False
    size_estimate = results["resultSizeEstimate"]
    print(f"next_token {next_token}, size estimate: {size_estimate}")
    for msg in messages:
      msg_dict = gmail_messages.get(userId='me', id=msg['id'], format='raw').execute()
      emails.append(msg_dict)
    count += len(emails)
    if len(emails) > CUT_OFF:
      has_next_token = False
      next_token = None
except HttpError as error:
  print(f'An error occurred: {error}')

In [ ]:
! pip install bs4 

In [ ]:
from bs4 import BeautifulSoup
import base64

# Function to remove tags
def remove_tags(html):
 
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [ ]:
def extract_header(headers, header_name:str) -> str:
  for h in headers:
    if h["name"] == header_name:
      return h["value"]

In [ ]:
len(emails)

In [ ]:
import email
from email import policy
from email.parser import Parser
from io import BytesIO
import io

parser = Parser(policy=policy.default)

# reference content dictionary
content_dictionary = {
}

counter = 0
for email in emails:
  msg_str = base64.urlsafe_b64decode(email['raw'].encode('ASCII'))
  
  subject = from_email = to_email = None
  msg = parser.parsestr(msg_str.decode('utf-8'))
  for key in msg.keys():
    if key == "Subject":
      subject = msg.get_all("Subject")
    if key == "From":
      from_email = msg.get_all("From")
    if key == "To":
      to_email = msg.get_all("To")
  print(f"from: {from_email}, subject: {subject}")
  for part in msg.walk():
     if part.get_content_type() == "text/html":
       content = part.get_content()
       clear_content = remove_tags(content)
       content_dictionary[counter] = {
           "id": email["id"],
           "subject": subject,
           "from": from_email,
           "to": to_email,
           "body": clear_content,
           "snippet": email["snippet"]
       }
       counter += 1

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
len(content_dictionary)

In [ ]:
corpus = [c["body"] for c in content_dictionary.values()]
print(f"embedding {len(corpus)} sentences")
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [ ]:
# Query sentences:
queries = ['where to hire a wall fixing guy in Salt Lake City?']

In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(1, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    # cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    # top_results = torch.topk(cos_scores, k=top_k)

    # print("\n\n======================\n\n")
    # print("Query:", query)
    # print("\nTop 5 most similar sentences in corpus:")

    # for score, idx in zip(top_results[0], top_results[1]):
    #     print(corpus[idx], "(Score: {:.4f})".format(score))
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=2)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        id = hit['corpus_id']
        item = content_dictionary[id]
        print(f"query: {query}, subject: {item['subject']}, {item['snippet']} https://mail.google.com/mail/u/2/#all/{item['id']}")
        # print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """